In [1]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus

In [2]:
pd.set_option('display.max_columns', None)

## Skirt 리뷰 전처리

In [279]:
df_skirt = pd.read_csv('./musinsa_review_skirt.csv')

In [280]:
df_skirt.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159cm,60kg,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,1911322,0,1,158cm,54kg,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,1911322,0,1,159cm,44kg,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


In [281]:
df_skirt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12391 entries, 0 to 12390
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   userName       12375 non-null  object
 1   date           12391 non-null  object
 2   goodsNo        12391 non-null  int64 
 3   userSexMen     12391 non-null  int64 
 4   userSexWomen   12391 non-null  int64 
 5   userHeight     12391 non-null  object
 6   userWeight     12391 non-null  object
 7   goodsSize      12391 non-null  object
 8   reviewContent  12391 non-null  object
 9   reviewImg      3049 non-null   object
 10  reviewStyle    12391 non-null  int64 
 11  size           12391 non-null  int64 
 12  bright         12391 non-null  int64 
 13  color          12391 non-null  int64 
 14  thickness      12391 non-null  int64 
 15  weightness     12391 non-null  int64 
 16  touch          12391 non-null  int64 
 17  helpNo         12391 non-null  int64 
 18  styleLikeNo    12391 non-n

In [282]:
df_skirt.shape

(12391, 19)

### 탈퇴 인원 제거

In [283]:
df_skirt.dropna(subset="userName", axis=0, inplace=True)
df_skirt = df_skirt.reset_index(drop=True)

In [284]:
df_skirt.shape

(12375, 19)

### Date를 년, 월, 일로 등분

In [285]:
df_skirt.dropna(subset="date", axis=0, inplace=True)

In [286]:
df_skirt.shape

(12375, 19)

In [287]:
df_skirt.insert(2, "dateDay", 0, True)
df_skirt.insert(2, "dateMonth", 0, True)
df_skirt.insert(2, "dateYear", 0, True)

In [288]:
df_skirt.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [289]:
df_skirt['dateYear'] = df_skirt.apply(lambda x: x['date'].split('.')[0], axis=1)
df_skirt['dateMonth'] = df_skirt.apply(lambda x: x['date'].split('.')[1], axis=1)
df_skirt['dateDay'] = df_skirt.apply(lambda x: x['date'].split('.')[2], axis=1)

In [290]:
df_skirt.head()

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2021,10,14,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2021,08,22,2029080,0,1,159cm,60kg,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2021,08,12,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,2021,08,06,1911322,0,1,158cm,54kg,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,2021,07,07,1911322,0,1,159cm,44kg,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


In [291]:
df_skirt = df_skirt.drop('date', axis=1)

In [292]:
df_skirt.dateYear = df_skirt.dateYear.astype('int')
df_skirt.dateMonth = df_skirt.dateMonth.astype('int')
df_skirt.dateDay = df_skirt.dateDay.astype('int')

### goodsNo

In [293]:
df_skirt.dropna(subset="goodsNo", axis=0, inplace=True)

In [294]:
df_skirt.shape

(12375, 21)

### 성별

In [295]:
df_skirt[df_skirt.userSexMen == 1].shape

(192, 21)

In [296]:
df_skirt[df_skirt.userSexWomen==1].shape

(12183, 21)

In [297]:
192+12183

12375

### 키와 몸무게

In [298]:
df_skirt.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [299]:
df_skirt.shape

(12375, 21)

In [300]:
df_skirt.userHeight = df_skirt.userHeight.apply(lambda x: x.replace("cm", ""))
df_skirt.userHeight = df_skirt.userHeight.astype('int64')

In [301]:
df_skirt.userWeight = df_skirt.userWeight.apply(lambda x: x.replace("kg", ""))
df_skirt.userWeight = df_skirt.userWeight.astype('int64')

### 이미지

In [302]:
df_skirt.reviewStyle.value_counts()

0    9328
1    3047
Name: reviewStyle, dtype: int64

### 사이즈

In [303]:
sub = df_skirt[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', sub.index[:], True)
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [304]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [305]:
for i in range(2654):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
0,2029080,KHAKI/FREE,2029080,0
1,1911322,M,1911322,1
2,1911322,S,1911322,2
3,1625877,FREE,1625877,3
4,1586158,L(28-30),1586158,4
...,...,...,...,...
2649,2117256,S,2117256,2649
2650,2117256,M,2117256,2650
2651,2117256,L,2117256,2651
2652,2340070,M,2340070,2652


In [306]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2654 entries, 0 to 2653
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     2654 non-null   object
 1   goodsSize   2654 non-null   object
 2   newGoodsNo  2654 non-null   object
 3   index       2654 non-null   object
dtypes: object(4)
memory usage: 83.1+ KB


In [307]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
0,2029080,KHAKI/FREE,20290800,0
1,1911322,M,19113221,1
2,1911322,S,19113222,2
3,1625877,FREE,16258773,3
4,1586158,L(28-30),15861584,4
...,...,...,...,...
2649,2117256,S,21172562649,2649
2650,2117256,M,21172562650,2650
2651,2117256,L,21172562651,2651
2652,2340070,M,23400702652,2652


In [308]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [309]:
df_skirt

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021,10,14,2029080,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021,8,22,2029080,0,1,159,60,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021,8,12,2029080,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021,8,6,1911322,0,1,158,54,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021,7,7,1911322,0,1,159,44,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,뉴비_54bed8fe,2022,3,6,2340070,0,1,160,48,M,"['보통 여기 치마는 m 사면 무난하게 맞았는데', <br/>, '얘는 좀 허리가 ...",NaN,0,1,2,2,0,0,0,0,0
12371,얼래프,2022,3,3,2340070,0,1,163,47,S,['패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ'],NaN,0,1,2,2,0,0,0,0,0
12372,교촌한입만,2022,2,25,2340070,0,1,160,50,S,"['엉덩이 있는 편인데 S 잘맞아요', <br/>, '허리는 25-26 정도인데 적...",NaN,0,2,1,1,0,0,0,0,0
12373,야옹웅,2022,2,18,2340070,0,1,158,50,M,['실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박'],NaN,0,2,2,1,0,0,0,0,0


#### month 당 사는 옷 수

In [310]:
df_skirt.insert(5, 'month12', 0, True)
df_skirt.insert(5, 'month11', 0, True)
df_skirt.insert(5, 'month10', 0, True)
df_skirt.insert(5, 'month9', 0, True)
df_skirt.insert(5, 'month8', 0, True)
df_skirt.insert(5, 'month7', 0, True)
df_skirt.insert(5, 'month6', 0, True)
df_skirt.insert(5, 'month5', 0, True)
df_skirt.insert(5, 'month4', 0, True)
df_skirt.insert(5, 'month3', 0, True)
df_skirt.insert(5, 'month2', 0, True)
df_skirt.insert(5, 'month1', 0, True)

In [311]:
df_skirt.month1 = df_skirt.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_skirt.month2 = df_skirt.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_skirt.month3 = df_skirt.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_skirt.month4 = df_skirt.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_skirt.month5 = df_skirt.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_skirt.month6 = df_skirt.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_skirt.month7 = df_skirt.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_skirt.month8 = df_skirt.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_skirt.month9 = df_skirt.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_skirt.month10 = df_skirt.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_skirt.month11 = df_skirt.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_skirt.month12 = df_skirt.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [315]:
d = df_skirt.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

,,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
goodsNo,goodsSize,,,,,,,,,,,,
387628,FREE/PINK,0,0,0,0,0,0,0,0,1,0,0,0
387631,FREE/BLUE,0,0,0,0,0,0,0,1,0,0,0,0
387632,FREE/NAVY,0,0,1,0,0,0,0,0,0,0,0,0
407626,FREE,0,0,1,0,0,0,0,0,0,0,0,0
463655,FREE,0,0,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389731,FREE,0,0,1,0,0,0,0,0,0,0,0,0
2391576,S,0,0,1,0,0,0,0,0,0,0,0,0
2391598,S,0,0,1,0,0,0,0,0,0,0,0,0


#### 키, 몸무게 평균 구하기

In [313]:
c = df_skirt.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,387628,FREE/PINK,160.0,47.0,0.0,1.0
1,387631,FREE/BLUE,158.0,46.0,0.0,1.0
2,387632,FREE/NAVY,163.0,50.0,0.0,1.0
3,407626,FREE,161.0,54.0,0.0,1.0
4,463655,FREE,150.0,50.0,0.0,1.0
...,...,...,...,...,...,...
2649,2389731,FREE,165.0,55.0,0.0,1.0
2650,2391576,S,161.0,46.0,0.0,1.0
2651,2391598,S,158.0,46.0,0.0,1.0
2652,2394978,M,160.0,55.0,0.0,1.0


In [316]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub

,goodsNo,goodsSize,newGoodsNo,index
1116,387628,FREE/PINK,38762801116,1116
1604,387631,FREE/BLUE,38763101604,1604
1044,387632,FREE/NAVY,38763201044,1044
740,407626,FREE,4076260740,740
465,463655,FREE,4636550465,465
...,...,...,...,...
2219,2389731,FREE,23897312219,2219
2318,2391576,S,23915762318,2318
1607,2391598,S,23915981607,1607
371,2394978,M,2394978371,371


In [317]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2649,0,0,1,0,0,0,0,0,0,0,0,0
2650,0,0,1,0,0,0,0,0,0,0,0,0
2651,0,0,1,0,0,0,0,0,0,0,0,0
2652,0,0,1,0,0,0,0,0,0,0,0,0


In [318]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,387628,FREE/PINK,38762801116,1116,162.702128,51.744681,0.042553,0.957447,9,10,0,1,0,0,0,0,1,3,11,12
1,387631,FREE/BLUE,38763101604,1604,165.000000,50.000000,0.000000,1.000000,0,0,2,0,0,0,0,0,0,0,0,0
2,387632,FREE/NAVY,38763201044,1044,163.000000,50.000000,0.000000,1.000000,2,0,0,0,0,0,0,0,0,0,0,0
3,407626,FREE,4076260740,740,165.000000,52.000000,0.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0
4,463655,FREE,4636550465,465,161.733333,53.933333,0.066667,0.933333,0,2,0,0,1,3,2,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,2389731,FREE,23897312219,2219,155.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,0,1,0,0
2650,2391576,S,23915762318,2318,159.625000,47.750000,0.125000,0.875000,4,3,1,0,0,0,0,0,0,0,0,0
2651,2391598,S,23915981607,1607,167.294118,65.647059,0.000000,1.000000,0,0,0,0,1,4,0,0,4,6,1,1
2652,2394978,M,2394978371,371,164.000000,52.250000,0.000000,1.000000,0,0,0,0,0,0,0,0,0,0,3,1


In [319]:
df_skirt.insert(4, 'newClothNo', 0, True)
df_skirt.head()

,userName,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021,10,14,0,2029080,0,0,0,0,0,0,0,0,0,1,0,0,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021,8,22,0,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,159,60,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021,8,12,0,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021,8,6,0,1911322,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,54,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021,7,7,0,1911322,0,0,0,0,0,0,1,0,0,0,0,0,0,1,159,44,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


In [320]:
# sub에서 goodsNo와 size를 통해 동일하면 newGoodsNo를 newClothNo에 대입

for i in df_skirt.itertuples():
    for j in sub.itertuples():
        if i.goodsNo == j.goodsNo and i.goodsSize==j.goodsSize:
            df_skirt.at[i.Index, 'newClothNo'] = j.newGoodsNo
df_skirt

,userName,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021,10,14,20290800,2029080,0,0,0,0,0,0,0,0,0,1,0,0,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021,8,22,20290800,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,159,60,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021,8,12,20290800,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021,8,6,19113221,1911322,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,54,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021,7,7,19113222,1911322,0,0,0,0,0,0,1,0,0,0,0,0,0,1,159,44,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,뉴비_54bed8fe,2022,3,6,23400702652,2340070,0,0,1,0,0,0,0,0,0,0,0,0,0,1,160,48,M,"['보통 여기 치마는 m 사면 무난하게 맞았는데', <br/>, '얘는 좀 허리가 ...",NaN,0,1,2,2,0,0,0,0,0
12371,얼래프,2022,3,3,23400702653,2340070,0,0,1,0,0,0,0,0,0,0,0,0,0,1,163,47,S,['패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ'],NaN,0,1,2,2,0,0,0,0,0
12372,교촌한입만,2022,2,25,23400702653,2340070,0,1,0,0,0,0,0,0,0,0,0,0,0,1,160,50,S,"['엉덩이 있는 편인데 S 잘맞아요', <br/>, '허리는 25-26 정도인데 적...",NaN,0,2,1,1,0,0,0,0,0
12373,야옹웅,2022,2,18,23400702652,2340070,0,1,0,0,0,0,0,0,0,0,0,0,0,1,158,50,M,['실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박'],NaN,0,2,2,1,0,0,0,0,0


In [323]:
sub.to_csv('skirt_size.csv',
                 sep=',',
                 float_format = '%.1f',
                 encoding = 'utf-8-sig')

### Contents

In [324]:
print(df_skirt.reviewContent[0])
print(df_skirt.reviewContent[1].split(","))

['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용']
["['생각보다 짧은 치마'", ' <br/>', " '제 체격에는 좀 작게 느껴질만한 사이즈'", ' <br/>', " '디자인은 되게 예쁨']"]


In [325]:
cnt = 0
result = []
for i in df_skirt.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

2


In [326]:
df_skirt[df_skirt.reviewContent=='[\'데님 색이 너무 맘에 들어서 구매했습니다!\', <br/>, \'키가 작아서 길이가 너무 길면 어떡하나 싶었는데 길이도 잘 맞고 허리도 너무 잘 맞았습니다>_\', \'!</div\', \'\\n\', <div class="review-evaluation">\n<ul class="review-evaluation__list">\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\n<li class="review-evaluation__item">밝기 <span>보통이에요</span></li>\n<li class="review-evaluation__item">색감 <span>보통이에요</span></li>\n</ul>\n</div>, \'\\n\', \' 이미지 \', \'\\n\', <div class="review-content-photo">\n<div class="review-content-photo__wrap">\n<ul class="review-content-photo__list">\n<li class="review-content-photo__item" data-img-index="0" style="background-image:url(\'//image.msscdn.net/data/estimate/1618698_0/gallery_607d08ce95b27.jpg.list\');">\n<img alt="밀리언코르(MILLIONCOR) [DANA 8130]커버밴드 A라인 데님 스커트(블랙그레이) 후기" src="//image.msscdn.net/data/estimate/1618698_0/gallery_607d08ce95b27.jpg.view"/>\n</li>\n</ul>\n</div>\n</div>, \'\\n\', \' 이모티콘 \', \'\\n\', \' SNS 공유시 필요한 데이터 \', \'\\n\', <span class="p_name" style="display: none;">[DANA 8130]커버밴드 A라인 데님 스커트(블랙그레이)</span>, \'\\n\']']

,userName,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
1184,여신예리,2021,4,19,1618698367,1618698,0,0,0,1,0,0,0,0,0,0,0,0,0,1,152,53,M,"['데님 색이 너무 맘에 들어서 구매했습니다!', <br/>, '키가 작아서 길이가...",https://image.msscdn.net/data/estimate/1618698...,1,2,2,2,0,0,0,0,0


In [327]:
df_skirt[df_skirt.reviewContent=='[\'허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같애요!! 예뻐요~>\', \'!</div\', \'\\n\', <div class="review-evaluation">\n<ul class="review-evaluation__list">\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\n<li class="review-evaluation__item">밝기 <span>보통이에요</span></li>\n<li class="review-evaluation__item">색감 <span>보통이에요</span></li>\n</ul>\n</div>, \'\\n\', \' 이미지 \', \'\\n\', \' 이모티콘 \', \'\\n\', \' SNS 공유시 필요한 데이터 \', \'\\n\', <span class="p_name" style="display: none;">Inside-Out Kangaroo Skirt Melange-Reverse</span>, \'\\n\']']

,userName,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
11323,써드3,2021,7,23,17887982306,1788798,0,0,0,0,0,0,1,0,0,0,0,0,0,1,163,50,S,['허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같...,NaN,0,2,2,2,0,0,0,0,0


In [328]:
df_skirt.loc[1186, 'reviewContent'] = result[0].split('</div')[0][:]
df_skirt.loc[11338, 'reviewContent'] = result[1].split('</div')[0][:]

In [329]:
print(df_skirt.loc[1186].reviewContent)
print(df_skirt.loc[11338].reviewContent)

['데님 색이 너무 맘에 들어서 구매했습니다!', <br/>, '키가 작아서 길이가 너무 길면 어떡하나 싶었는데 길이도 잘 맞고 허리도 너무 잘 맞았습니다>_', '!
['허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같애요!! 예뻐요~>', '!


In [330]:
df_skirt.reviewContent = df_skirt.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [331]:
print(df_skirt.reviewContent[0])
print(df_skirt.reviewContent[1])

['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용']
['생각보다 짧은 치마', '제 체격에는 좀 작게 느껴질만한 사이즈', '디자인은 되게 예쁨']


### 그 외

In [332]:
df_skirt['size'].value_counts()

2    9747
1    1872
3     756
Name: size, dtype: int64

In [333]:
df_skirt.bright.value_counts()

2    10196
3     1211
1      968
Name: bright, dtype: int64

In [334]:
df_skirt.color.value_counts()

2    8745
1    3468
3     162
Name: color, dtype: int64

In [335]:
df_skirt.thickness.value_counts()

0    12372
2        2
1        1
Name: thickness, dtype: int64

In [336]:
df_skirt.weightness.value_counts()

0    12375
Name: weightness, dtype: int64

In [337]:
df_skirt.touch.value_counts()

0    12375
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [340]:
df_skirt.head()

,userName,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021,10,14,20290800,2029080,0,0,0,0,0,0,0,0,0,1,0,0,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021,8,22,20290800,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021,8,12,20290800,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021,8,6,19113221,1911322,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021,7,7,19113222,1911322,0,0,0,0,0,0,1,0,0,0,0,0,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


## User 데이터 생성 (unique한 유저 정보로 수정)

In [362]:
user_skirt = df_skirt[['userName', 'userSexMen', 'userSexWomen']]
user_skirt = user_skirt.drop_duplicates(subset=['userName'])
user_skirt.sort_values('userName', inplace=True)
user_skirt.index = range(len(user_skirt))

In [363]:
mo = df_skirt.groupby('userName').sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
mo.sort_values('userName', inplace=True)
mo.index = range(len(mo))

In [364]:
hw = df_skirt.groupby('userName').mean()[['userHeight', 'userWeight']]
hw.sort_values('userName', inplace=True)
hw.index = range(len(hw))

In [365]:
user_skirt = pd.concat([user_skirt, hw, mo], axis=1)
user_skirt

,userName,userSexMen,userSexWomen,userHeight,userWeight,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,!@@!,0,1,160.0,55.0,0,0,0,0,0,0,0,0,0,0,0,1
1,#999,0,1,160.0,49.0,0,0,0,1,0,0,0,0,0,0,0,1
2,#Mini._Squeak,0,1,155.0,50.0,1,0,0,0,0,0,0,0,0,0,0,0
3,#_,0,1,161.0,44.0,0,0,0,0,0,0,0,0,0,0,0,1
4,*ㅁ*,0,1,164.0,47.0,4,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8292,히힛우히힛후힛,0,1,157.0,55.0,0,0,0,0,0,0,0,0,0,0,2,0
8293,힘빠지는 해파리,0,1,152.0,45.0,0,1,0,0,0,0,0,0,0,0,0,0
8294,힝힝깅,0,1,169.0,48.0,0,0,0,0,0,0,0,0,0,1,0,0
8295,힣히힣힣힣,0,1,160.0,48.0,1,0,0,0,0,0,0,0,0,0,0,0


In [366]:
user_skirt[user_skirt.userName=="사이먼0112"]

,userName,userSexMen,userSexWomen,userHeight,userWeight,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
5753,사이먼0112,0,1,156.0,48.0,0,0,2,0,0,0,0,0,0,0,0,0


In [371]:
user_skirt[user_skirt.userName=='k_hnnj'].index

Int64Index([1382], dtype='int64')

## Transaction 데이터 생성(전체 리뷰인 경우로 수정)

In [379]:
transaction_data = df_skirt[['userName', 'newClothNo']]
transaction_data.insert(0, 'userNo', 0, True)

In [380]:
transaction_data.userNo = transaction_data.apply(lambda x: user_skirt[user_skirt.userName==x.userName].index[0], axis=1)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_10108\797650514.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_data.userNo = transaction_data.apply(lambda x: user_skirt[user_skirt.userName==x.userName].index[0], axis=1)


In [381]:
transaction_data = transaction_data.drop('userName', axis=1)
transaction_data

,userNo,newClothNo
0,1382,20290800
1,2130,20290800
2,1382,20290800
3,5195,19113221
4,5593,19113222
...,...,...
12370,3217,23400702652
12371,6421,23400702653
12372,2315,23400702653
12373,6359,23400702652
